***GENERATED CODE FOR modelregress PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Product type', 'transformation_label': 'String Indexer'}], 'feature': 'Product type', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '80', 'mean': '', 'stddev': '', 'min': 'cosmetics', 'max': 'skincare', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Product type'}, {'feature_label': 'Product type', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Product type')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'SKU', 'transformation_label': 'String Indexer'}], 'feature': 'SKU', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '80', 'mean': '', 'stddev': '', 'min': 'SKU0', 'max': 'SKU9', 'missing': '0', 'distinct': '80'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'SKU'}, {'feature_label': 'SKU', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('SKU')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Customer demographics', 'transformation_label': 'String Indexer'}], 'feature': 'Customer demographics', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '80', 'mean': '', 'stddev': '', 'min': 'Female', 'max': 'Unknown', 'missing': '0', 'distinct': '4'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Customer demographics'}, {'feature_label': 'Customer demographics', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Customer demographics')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Shipping carriers', 'transformation_label': 'String Indexer'}], 'feature': 'Shipping carriers', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '80', 'mean': '', 'stddev': '', 'min': 'Carrier A', 'max': 'Carrier C', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Shipping carriers'}, {'feature_label': 'Shipping carriers', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Shipping carriers')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Supplier name', 'transformation_label': 'String Indexer'}], 'feature': 'Supplier name', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '80', 'mean': '', 'stddev': '', 'min': 'Supplier 1', 'max': 'Supplier 5', 'missing': '0', 'distinct': '5'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Supplier name'}, {'feature_label': 'Supplier name', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Supplier name')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Location', 'transformation_label': 'String Indexer'}], 'feature': 'Location', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '80', 'mean': '', 'stddev': '', 'min': 'Bangalore', 'max': 'Mumbai', 'missing': '0', 'distinct': '5'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Location'}, {'feature_label': 'Location', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Location')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Inspection results', 'transformation_label': 'String Indexer'}], 'feature': 'Inspection results', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '80', 'mean': '', 'stddev': '', 'min': 'Fail', 'max': 'Pending', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Inspection results'}, {'feature_label': 'Inspection results', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Inspection results')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Transportation modes', 'transformation_label': 'String Indexer'}], 'feature': 'Transportation modes', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '80', 'mean': '', 'stddev': '', 'min': 'Air', 'max': 'Sea', 'missing': '0', 'distinct': '4'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Transportation modes'}, {'feature_label': 'Transportation modes', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Transportation modes')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Routes', 'transformation_label': 'String Indexer'}], 'feature': 'Routes', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '80', 'mean': '', 'stddev': '', 'min': 'Route A', 'max': 'Route C', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Routes'}, {'feature_label': 'Routes', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Routes')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from sklearn.model_selection import train_test_split
from tpot import TPOTRegressor
import pyspark


def functionRegression(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = sparkDF.toPandas()
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTRegressor(verbosity=3, generations=10, max_time_mins=5,
                              n_jobs=-1, random_state=25, population_size=15, use_dask=True)
    tpotModel.fit(X_train, y_train)
    display(" Error rate of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run modelregressHooks.ipynb
try:
	#sourcePreExecutionHook()

	supplychaindata = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/supply_chain_data.csv', 'filename': 'supply_chain_data.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line', 'domain': 'http://172.31.59.158', 'port': '40070', 'dirPath': '/FileStore/platform', 'server_url': '/numtraPlatform/NumtraPlatformV3/uploads/platform/'}")
	#sourcePostExecutionHook(supplychaindata)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run modelregressHooks.ipynb
try:
	#transformationPreExecutionHook()

	modelregressautofe = TransformationMain.run(supplychaindata,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "Product type", "transformation_label": "String Indexer"}], "feature": "Product type", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "80", "mean": "", "stddev": "", "min": "cosmetics", "max": "skincare", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Product type"}, {"transformationsData": [{"feature_label": "SKU", "transformation_label": "String Indexer"}], "feature": "SKU", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "80", "mean": "", "stddev": "", "min": "SKU0", "max": "SKU9", "missing": "0", "distinct": "80"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "SKU"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Price", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "80", "mean": "47.85", "stddev": "31.06", "min": "1.0", "max": "99.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Price"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Availability", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "80", "mean": "47.3", "stddev": "30.71", "min": "1", "max": "100", "missing": "0"}, "updatedLabel": "Availability"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Number of products sold", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "80", "mean": "455.34", "stddev": "299.73", "min": "8", "max": "996", "missing": "0"}, "updatedLabel": "Number of products sold"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Revenue generated", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "80", "mean": "5793.64", "stddev": "2701.6", "min": "1061.0", "max": "9866.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Revenue generated"}, {"transformationsData": [{"feature_label": "Customer demographics", "transformation_label": "String Indexer"}], "feature": "Customer demographics", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "80", "mean": "", "stddev": "", "min": "Female", "max": "Unknown", "missing": "0", "distinct": "4"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Customer demographics"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Stock levels", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "80", "mean": "45.98", "stddev": "32.57", "min": "0", "max": "100", "missing": "0"}, "updatedLabel": "Stock levels"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Lead times", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "80", "mean": "16.21", "stddev": "8.95", "min": "1", "max": "30", "missing": "0"}, "updatedLabel": "Lead times"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Order quantities", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "80", "mean": "49.77", "stddev": "26.11", "min": "1", "max": "96", "missing": "0"}, "updatedLabel": "Order quantities"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Shipping times", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "80", "mean": "5.62", "stddev": "2.67", "min": "1", "max": "10", "missing": "0"}, "updatedLabel": "Shipping times"}, {"transformationsData": [{"feature_label": "Shipping carriers", "transformation_label": "String Indexer"}], "feature": "Shipping carriers", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "80", "mean": "", "stddev": "", "min": "Carrier A", "max": "Carrier C", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Shipping carriers"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Shipping costs", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "80", "mean": "5.06", "stddev": "2.66", "min": "1.0", "max": "9.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Shipping costs"}, {"transformationsData": [{"feature_label": "Supplier name", "transformation_label": "String Indexer"}], "feature": "Supplier name", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "80", "mean": "", "stddev": "", "min": "Supplier 1", "max": "Supplier 5", "missing": "0", "distinct": "5"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Supplier name"}, {"transformationsData": [{"feature_label": "Location", "transformation_label": "String Indexer"}], "feature": "Location", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "80", "mean": "", "stddev": "", "min": "Bangalore", "max": "Mumbai", "missing": "0", "distinct": "5"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Location"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Lead time", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "80", "mean": "17.25", "stddev": "8.72", "min": "1", "max": "30", "missing": "0"}, "updatedLabel": "Lead time"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Production volumes", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "80", "mean": "561.76", "stddev": "269.67", "min": "104", "max": "985", "missing": "0"}, "updatedLabel": "Production volumes"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Manufacturing lead time", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "80", "mean": "14.78", "stddev": "8.82", "min": "1", "max": "30", "missing": "0"}, "updatedLabel": "Manufacturing lead time"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Manufacturing costs", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "80", "mean": "46.25", "stddev": "29.73", "min": "1.0", "max": "99.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Manufacturing costs"}, {"transformationsData": [{"feature_label": "Inspection results", "transformation_label": "String Indexer"}], "feature": "Inspection results", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "80", "mean": "", "stddev": "", "min": "Fail", "max": "Pending", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Inspection results"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Defect rates", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "80", "mean": "1.8", "stddev": "1.38", "min": "0.0", "max": "4.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Defect rates"}, {"transformationsData": [{"feature_label": "Transportation modes", "transformation_label": "String Indexer"}], "feature": "Transportation modes", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "80", "mean": "", "stddev": "", "min": "Air", "max": "Sea", "missing": "0", "distinct": "4"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Transportation modes"}, {"transformationsData": [{"feature_label": "Routes", "transformation_label": "String Indexer"}], "feature": "Routes", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "80", "mean": "", "stddev": "", "min": "Route A", "max": "Route C", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Routes"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Costs", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "80", "mean": "512.31", "stddev": "247.5", "min": "110.0", "max": "997.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Costs"}]}))

	#transformationPostExecutionHook(modelregressautofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run modelregressHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionRegression(modelregressautofe, ["Price", "Availability", "Revenue generated", "Stock levels", "Lead times", "Order quantities", "Shipping times", "Shipping costs", "Lead time", "Production volumes", "Manufacturing lead time", "Manufacturing costs", "Defect rates", "Costs", "Product type_stringindexer", "SKU_stringindexer", "Customer demographics_stringindexer", "Shipping carriers_stringindexer", "Supplier name_stringindexer", "Location_stringindexer", "Inspection results_stringindexer", "Transportation modes_stringindexer", "Routes_stringindexer"], "Number of products sold")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML ['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    df = df[columnNames]
    R2 = np.round(sklearn.metrics.r2_score(y_test, y_predicted), 1)
    Mean_Squared_Error = np.round(sklearn.metrics.mean_squared_error(y_test, y_predicted), 1)
    Mean_Absolute_Error = np.round(sklearn.metrics.mean_absolute_error(y_test, y_predicted), 1)
    display(" R2 score of Prediction on test data    : %s"%R2)
    display(" Mean Squared Error of Prediction on test data    : %s"%Mean_Squared_Error)
    display(" Mean Absolute Error of Prediction on test data   : %s"%Mean_Absolute_Error)
    display(df.head())
except Exception as ex:
    logging.error(ex)

spark.stop()

